In [2]:
import mysql.connector as connector
from mysql.connector import Error
import pandas as pd
from datetime import date

In [3]:
def create_connection(host="localhost", user="root", password="P@ssw0rd123!", database="LittleLemonDB"):
    try:
        conn = connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        if conn.is_connected():
            db_info = conn.get_server_info()
            print(f"Connected to MySQL Server Version: {db_info}")
            print(f"Connected to database: {database}")
            return conn
            
    except Error as e:
        print(f"Error Connection to MySQL: {e}")
        return None




In [39]:
def show_tables():
    conn = create_connection()
    if not conn:
        return

    try:
        cursor = conn.cursor()
        show_table_query = "SHOW TABLES"
        cursor.execute(show_table_query)
        tables = cursor.fetchall()
        col = [desc[0] for desc in cursor.description]
        print(col)
        for table in tables:
            print(f"{table[0]}")

       
    finally:
        cursor.close()
        conn.close()

show_tables()

Connected to MySQL Server Version: 8.0.43-0ubuntu0.24.04.2
Connected to database: LittleLemonDB
['Tables_in_LittleLemonDB']
Bookings
Customers
Menu
MenuItems
OrderDeliveryStatus
Orders
OrdersView
Staff


/tmp/ipykernel_10909/1166277589.py:11: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = conn.get_server_info()


In [44]:
def get_max_quantity():
    conn = create_connection()
    if not conn:
        return None

    try:
        cursor = conn.cursor()
        cursor.callproc("GetMaxQuantity")

        max_qty = None
        for result in cursor.stored_results():
            row = result.fetchone()
            if row:
                max_qty = row[0]
        print(f"Maximum Quantity from Orders: {max_qty}")
        cursor.close()
        return max_qty

    except Error as e:
        print(f"Error calling GetMaxQuantity: {e}")
        return None
    finally:
        conn.close()

result = get_max_quantity()

Connected to MySQL Server Version: 8.0.43-0ubuntu0.24.04.2
Connected to database: LittleLemonDB
Maximum Quantity from Orders: 6


/tmp/ipykernel_10909/1166277589.py:11: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = conn.get_server_info()
/tmp/ipykernel_10909/2878669932.py:11: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  for result in cursor.stored_results():


In [73]:
def add_booking_python(booking_id, customer_id, booking_date, table_number):
    conn = create_connection()
    if not conn:
        return None
    
    try:
        cursor = conn.cursor()
        
        params = [booking_id, customer_id, booking_date, table_number]
        cursor.callproc("AddBooking", params)
        
        results = []
        for r in cursor.stored_results():
            rows = r.fetchall()
            results.extend(rows)
        
        conn.commit()
        
        print("Stored procedure output:")
        for row in results:
            print(row)
        
        cursor.close()
        return results
    
    except Error as e:
        print("Error calling AddBooking:", e)
        conn.rollback()
        return None
    finally:
        conn.close()

add_booking_python(50,1,"2025-10-20",5)

/tmp/ipykernel_10909/1166277589.py:11: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = conn.get_server_info()
/tmp/ipykernel_10909/2622908364.py:13: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  for r in cursor.stored_results():


Connected to MySQL Server Version: 8.0.43-0ubuntu0.24.04.2
Connected to database: LittleLemonDB
Stored procedure output:
('Booking Successfully Added!',)
(50, 1, 'John Smith', datetime.date(2025, 10, 20), 5, datetime.datetime(2025, 11, 18, 15, 36, 12))


[('Booking Successfully Added!',),
 (50,
  1,
  'John Smith',
  datetime.date(2025, 10, 20),
  5,
  datetime.datetime(2025, 11, 18, 15, 36, 12))]

In [67]:
def show_addbooking_definition():
    conn = create_connection()
    if not conn:
        return
    
    cursor = None
    try:
        cursor = conn.cursor()
        cursor.execute("SHOW CREATE PROCEDURE AddBooking")
        row = cursor.fetchone()
        print(row[2])  # This prints the CREATE PROCEDURE SQL
    finally:
        if cursor:
            cursor.close()
        conn.close()

show_addbooking_definition()

Connected to MySQL Server Version: 8.0.43-0ubuntu0.24.04.2
Connected to database: LittleLemonDB
CREATE DEFINER=`admin`@`%` PROCEDURE `AddBooking`(
	IN booking_id INT,
    IN customer_id INT,
    IN booking_date DATE,
    IN table_number INT
)
BEGIN

	DECLARE customer_exists INT DEFAULT 0;
	DECLARE booking_exists INT DEFAULT 0;
	DECLARE table_booked INT DEFAULT 0;
	DECLARE customer_name VARCHAR(255) DEFAULT '';

	SELECT COUNT(*) INTO customer_exists
	FROM Customers
	WHERE CustomerID = customer_id;
    
    IF customer_exists > 0 THEN
		SELECT FullName INTO customer_name
        FROM Customers
        WHERE CustomerID = customer_id;
    END IF;

	SELECT COUNT(*) INTO booking_exists
	FROM Bookings
	WHERE BookingID = booking_id;

	SELECT COUNT(*) INTO table_booked
	FROM Bookings
	WHERE BookingDate = booking_date 
	AND TableNumber = table_number;

	IF customer_exists = 0 THEN
		SELECT CONCAT('Error: CustomerID ', customer_id, ' dose not exist!') AS 'Status',
        'Please register the cus

/tmp/ipykernel_10909/1166277589.py:11: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = conn.get_server_info()


In [5]:
def update_booking(booking_id, booking_date):

    conn = create_connection()
    if not conn:
        return 0

    try:
        cursor = conn.cursor()
        params = [booking_id, booking_date]
        cursor.callproc("UpdateBooking", params)
        rows_updated = 0
        for result in cursor.stored_results():
            row = result.fetchone()
            if row:
                rows_updated = row[0]

        conn.commit()
        if rows_updated > 0:
            print("Booking updated successfully!")
            print(f"Booking ID: {booking_id}")
            print(f"New Date: {booking_date}")
            print(f"Rows Updated: {rows_updated}")

        else:
            print(f"No booking found with ID: {booking_id}")

        cursor.close()
        return rows_updated

    except Error as e:
        print(f"Error calling updating: {e}")
        conn.rollback()
        return 0

    finally:
        conn.close()

updated = update_booking(1, "2025-12-5")

/tmp/ipykernel_5271/1166277589.py:11: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = conn.get_server_info()


Connected to MySQL Server Version: 8.0.43-0ubuntu0.24.04.2
Connected to database: LittleLemonDB


/tmp/ipykernel_5271/4105758098.py:12: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  for result in cursor.stored_results():


Booking updated successfully!
Booking ID: 1
New Date: 2025-12-5
Rows Updated: 1


In [14]:
def cancel_booking(booking_id):
    conn = create_connection()
    if not conn:
        return 0

    try:
        cursor = conn.cursor()
        cursor.callproc("CancelBooking", [booking_id])
        rows_deleted = 0
        results = list(cursor.stored_results())

        if results:
            last_result = results[-1]
            row = last_result.fetchone()
            if row:
                try:
                    rows_deleted = int(row[0])
                except (ValueError, TypeError):
                    print(f"Error from stored procedure: {row[0]}")
                    rows_deleted = 0
                  
        conn.commit()

        if rows_deleted > 0:
            print("Booking cancelled successfully!")
            print(f"Booking ID: {booking_id}")
            print(f"Rows Deleted: {rows_deleted}")
        else:
            print(f"No booking found with ID: {booking_id}")

        cursor.close()
        return rows_deleted
    except Error as e:
        print(f"Error calling CancelBooking: {e}")
        conn.rollback()
        return 0

    finally: 
        conn.close()

deleted = cancel_booking(1)

/tmp/ipykernel_5271/1166277589.py:11: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = conn.get_server_info()
/tmp/ipykernel_5271/3401153329.py:10: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  results = list(cursor.stored_results())


Connected to MySQL Server Version: 8.0.43-0ubuntu0.24.04.2
Connected to database: LittleLemonDB
Booking cancelled successfully!
Booking ID: 1
Rows Deleted: 1


In [18]:
def view_booking_table():
    conn = create_connection()
    if not conn:
        return 0

    try:
        view_query = """
        SELECT 
            b.BookingID,
            b.BookingDate,
            b.TableNumber,
            c.CustomerID,
            c.FullName AS CustomerName,
            c.PhoneNumber
        FROM Bookings b
        JOIN Customers c ON b.CustomerID = c.CustomerID
        ORDER BY b.BookingDate DESC;
        """

        df = pd.read_sql(view_query, conn)
        display(df)
        return df

    except Error as e:
        print(f"Error: {e}")
        return 0

    finally:
        conn.close()

view_booking_table()

/tmp/ipykernel_5271/1166277589.py:11: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = conn.get_server_info()
/tmp/ipykernel_5271/4008137075.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(view_query, conn)


Connected to MySQL Server Version: 8.0.43-0ubuntu0.24.04.2
Connected to database: LittleLemonDB


,BookingID,BookingDate,TableNumber,CustomerID,CustomerName,PhoneNumber
0,51,2025-12-20,5,1,John Smith,555-0101
1,50,2025-10-20,5,1,John Smith,555-0101
2,9,2022-09-12,1,9,Robert Taylor,555-0109
3,8,2022-07-20,9,7,David Martinez,555-0107
4,7,2021-08-13,6,8,Lisa Anderson,555-0108
5,6,2021-05-16,4,5,Michael Brown,555-0105
6,5,2020-06-11,7,6,Sarah Wilson,555-0106
7,4,2020-04-16,2,4,Emily Davis,555-0104
8,3,2019-03-10,8,3,James Johnson,555-0103
9,2,2019-02-20,3,2,Maria Garcia,555-0102


,BookingID,BookingDate,TableNumber,CustomerID,CustomerName,PhoneNumber
0,51,2025-12-20,5,1,John Smith,555-0101
1,50,2025-10-20,5,1,John Smith,555-0101
2,9,2022-09-12,1,9,Robert Taylor,555-0109
3,8,2022-07-20,9,7,David Martinez,555-0107
4,7,2021-08-13,6,8,Lisa Anderson,555-0108
5,6,2021-05-16,4,5,Michael Brown,555-0105
6,5,2020-06-11,7,6,Sarah Wilson,555-0106
7,4,2020-04-16,2,4,Emily Davis,555-0104
8,3,2019-03-10,8,3,James Johnson,555-0103
9,2,2019-02-20,3,2,Maria Garcia,555-0102


In [28]:
def manage_booking(action, booking_id=None, booking_date=None, table_number=None, customer_id=None):
    conn = create_connection()
    if not conn:
        return 0

    try:
        cursor = conn.cursor()
        params = [action, booking_id, booking_date, table_number, customer_id]

        cursor.callproc("ManageBooking", params)

        results_list = []
        for results in cursor.stored_results():
            rows = results.fetchall()
            if rows:
                results_list.extend(rows)

        conn.commit

        print(f"ManageBooking - Action: {action}")

        if action == 'VIEW':
            if results_list:
                df = pd.DataFrame(results_list, columns=['BookingID', 'BookingDate', 'TableNumber', 'CustomerID'])
                print(df.to_string(index=False))
            else:
                print("No booking found")
        else:
            for row in results_list:
                print("Result: {row}")

        cursor.close()
        return results_list

    except Error as e:
        print(f"Error calling ManageBooking: {e}")
        conn.rollback()
        return 0

    finally:
        conn.close()

manage_booking('CANCEL', booking_id=2)


Connected to MySQL Server Version: 8.0.43-0ubuntu0.24.04.2
Connected to database: LittleLemonDB
ManageBooking - Action: CANCEL
Result: {row}


/tmp/ipykernel_5271/1166277589.py:11: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = conn.get_server_info()
/tmp/ipykernel_5271/3046357434.py:13: DeprecationWarning: Call to deprecated function stored_results. Reason: The property counterpart 'stored_results' will be added in a future release, and this method will be removed.
  for results in cursor.stored_results():


[('✅ Booking cancelled successfully! BookingID: 2',)]